In [2]:
# Permet de tout executer au lancement du notebook + conserver le notebook actif pendant 2h
from IPython.display import Javascript
from masquer import *
Javascript("""
function repeter(){
IPython.notebook.kernel.execute("a=1");
}
// execute a = 1 en python toutes les 8 minutes pendant 2h
let timerId = setInterval(() => repeter(), 4800);
setTimeout(() => { clearInterval(timerId); alert('fin de cession'); }, 7200000);

// Supprimer la taille limite pour la sortie d'une cellule
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
};
IPython.notebook.kernel.execute("url = '" + window.location + "'");

// Exécuter toutes les cellule du notebook
    require(
        ['base/js/namespace', 'jquery'], 
        function(jupyter, $) {
            
                
                jupyter.actions.call('jupyter-notebook:run-all-cells-below');
                jupyter.actions.call('jupyter-notebook:save-notebook');
                Jupyter.actions.call('jupyter-notebook:hide-header')

        }
    );""")

<IPython.core.display.Javascript object>

In [3]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
HTML("""<style>
h1 {
  font-family: 'Permanent Marker', cursive;
  text-align: center;
  color: red;
  
}
ol {
  list-style-position: inside;
  margin-left: 1em;
  list-style-position: outside;
}
h2 {
  font-family: 'Permanent Marker', cursive;
  color: blue;
}
h3 {
  font-family: 'Permanent Marker', cursive;

}
</style>""")

# CHAPITRE 4 - Révisions méthodes numériques d'intégration

Nous pouvons calculer l'approximation d'une intégrale en découpant la courbe en petites portions et en réalisant la somme des aires sous la courbe.

Considérons une fonction **f** qui à x associes f(x) que l'on souhaite intégrer sur un intervalle [a, b].

On découpe l'intervalle en **N** petits éléments de largeur dx de manière à avoir $x_0 = a$, $x_1 = a + dx$ etc....

## III. Méthode de simpson

On va maintenant approximer la fonction **f** à intégrer par un polynome de degré 2 de la forme $P(x) = a x^2 + b x + c$

Avec trois paramètres **a, b et c** à ajuster, on peut faire passer ce polynôme par les points $(x_i, f(x_i))$, $(x_{i+1}, f(x_{i+1}))$ et $(m, f(m))$ où $m = \frac{x_i + x_{i+1}}{2}$

Le polynôme **P(x)** sera différent pour chaque petit segment $[x_i, x_{i+1}]$, on peut le noter $P_i(x)$ par exemple.
Le polynôme $P_i(x) = f(x_i)\times \frac{(x - m)(x - x_{i+1})}{(x_i - m)(x_i - x_{i+1})} + f(m) \times \frac{(x - x_i)(x - x_{i+1})}{(m - x_i)(m - x_{i+1})} + f(x_{i+1})\times \frac{(x-x_i)(x-m)}{(x_{i+1}-x_i)(x_{i+1}-m)}$ 

respecte tous les critères.
En effet, $P_i(x_i) = f(x_i)$ car les deux derniers termes sont nuls et la fraction du premier est égal à un. De même, $P_i(m) = f(m)$ et $P_i(x_{i+1}) = f(x_{i+1})$. 



Ainsi $$\int_{a}^{b} f(x)dx \approx \sum_{i} \int_{x_i}^{x_{i+1}}P_i(x)dx$$


L'intégrale du polynôme $P_i(x)$ sur le segement $[x_i, x_{i+1}]$ peut se calculer exactement par:

$$\int_{x_i}^{x_{i+1}}P_i(x)dx = \frac{x_{i+1}-x_i}{6} \biggl[ f(x_i) + 4 f(m) + f(x_{i+1}) \biggr]$$.


Il ne reste plus qu'à faire la somme de ces différents terme pour obtenir l'approximation de l'intégrale.



1. Ecrire une fonction **simpson** qui prend comme argument **a, b, N et f** et qui renvoie l'approximation de l'intégrale de f sur le segment [a,b] par la méthode de simpson.
2. Comparez les résultats obtenus par cette méthode et la valeur exacte de l'intégrale pour la fonction qui à x associe $2.5 \times cos(2 \pi x / 0.1 + \pi/4)$ sur le segment [0, 1] pour différentes valeurs de N.
3. Comparez le résultat obtenu avec celui obtenu par la méthode des rectangles ou celle des trapèzes avec le même N.

In [4]:
from math import cos, pi

def f(x):
    return 2.5*cos(2*pi*x/0.22 + pi/4)

def xi(a,i,dx):
    return a + i*dx

def simpson(a,b,N,f):
    dx = (b-a)/N
    somme = 0
    for i in range(N):
        somme += (dx / 6) * (f(xi(a,i,dx)) + 4*f((xi(a,i,dx)+xi(a,i+1,dx))/2) + f(xi(a,i+1,dx)))
    return somme


In [5]:
import math
def f(x):
    return 2.5 * math.cos(2*math.pi*x / 0.22 + math.pi / 4)

import numpy as np
def f2(x):
    """
    x est un array
    """
    return 2.5 * np.cos(2*np.pi*x / 0.22 + np.pi / 4)

In [7]:
def rect_np(a,b,N,f):
    x = np.linspace(a,b,N+1)
    y = f(x) # array
    dx = (b-a)/N
    rects = dx * y
    rects = rects[:-1]
    return np.sum(rects)

In [6]:
def trapeze_np(a,b,N,f):
    x = np.linspace(a,b,N+1)
    y = f(x) # array
    dx = (b-a)/N
    trapezes = dx * 0.5*(y[1:]+y[:-1])
    return np.sum(trapezes)

In [8]:
import numpy as np
def simpson_np(a,b,N,f):
    x = np.linspace(a,b,2*N+1)
    y = f(x) # array
    dx = (b-a)/N
    surfaces = dx / 6 * (y[:-2:2]+y[2::2]+4*y[1:-1:2])
    return np.sum(surfaces)

In [9]:
def F(x):
    return 2.5*0.22/(2*math.pi) * math.sin(2*math.pi*x/0.22 + math.pi/4)

In [10]:
simpson(0.,1.,1000,f)

-0.13872456906848984

In [11]:
simpson_np(0.,1.,1000,f2)

-0.13872456906848976

In [12]:
F(1.)-F(0.)

-0.13872456903644192

In [13]:
trapeze_np(0.,1.,100000,f2)

-0.13872456809349662

In [15]:
rect_np(0.,1.,10000000,f2)

-0.1387244207418766